### Greedy Agent Game Type Selection

In [1]:
import sys
import os
src_path = os.path.abspath("src")
if src_path not in sys.path:
    sys.path.append(src_path)

import random
from tqdm import tqdm
from envs.deck import Deck
from envs.card import Suit
import utils

utils.seed_everything(42)

def game_type_probabilities(n: int):
    is_geschoben = False
    
    count_game_type = {
        "TOP_DOWN": 0,
        "BOTTOM_UP": 0,
        "ROSE": 0,
        "SCHILTE": 0,
        "EICHEL": 0,
        "SCHELLE": 0,
        "SCHIEBEN": 0
    }

    for _ in tqdm(range(n)):
        if not is_geschoben and random.random() < 0.14:
            count_game_type["SCHIEBEN"] += 1
            continue

        deck = Deck()
        hand = deck.cards[:9]
        hand.sort(key=lambda card: (card.suit.value, card.rank.value))
        #print(hand)

        # Choose top-down if many high cards
        high_cards = [card for card in hand if card.rank.value > 10]
        if len(high_cards) > 5:
            count_game_type["TOP_DOWN"] += 1
            continue

        # Choose bottom-up if many low cards
        low_cards = [card for card in hand if card.rank.value <= 9]
        if len(low_cards) > 5:
            count_game_type["BOTTOM_UP"] += 1
            continue

        # Choose trump for which it has the most cards
        suits = [card.suit for card in hand]
        suit_counts = {suit: suits.count(suit) for suit in set(suits)}
        max_count = max(suit_counts.values())
        max_suits = [suit for suit, count in suit_counts.items() if count == max_count]

        trump = random.choice(max_suits)

        if trump == Suit.ROSE:
            count_game_type["ROSE"] += 1
            continue
        elif trump == Suit.SCHILTE:
            count_game_type["SCHILTE"] += 1
            continue
        elif trump == Suit.EICHEL:
            count_game_type["EICHEL"] += 1
            continue
        elif trump == Suit.SCHELLE:
            count_game_type["SCHELLE"] += 1
            continue
        else:
            raise ValueError("Suit not recognized")
    
    game_type_probs = {game_type: count / n for game_type, count in count_game_type.items()}
    return game_type_probs


n = 1000000
game_type_probs = game_type_probabilities(n)

print(sum(game_type_probs.values()))
print(game_type_probs)

100%|██████████| 1000000/1000000 [00:18<00:00, 54112.11it/s]

1.0
{'TOP_DOWN': 0.105955, 'BOTTOM_UP': 0.105919, 'ROSE': 0.161873, 'SCHILTE': 0.161824, 'EICHEL': 0.162231, 'SCHELLE': 0.1619, 'SCHIEBEN': 0.140298}


### DQN Agent Game Type Selection

In [2]:
model_path = "models/FI_dqn_128-128_500000.pt"
hide_opponents_hands = False
    
from tqdm import tqdm
from agents.dqn_agent import DQN_Agent
from agents.greedy_agent import Greedy_Agent
from envs.jassenv import JassEnv

def game_type_probabilities_dqn(n: int):
    
    agent = DQN_Agent(player_id=0, 
                        team_id=0,
                        deterministic=True,
                        hide_opponents_hands=hide_opponents_hands,
                        hidden_sizes=[128, 128],
                        activation="relu")
    agent.load_model(model_path)
    
    players = [agent,
               Greedy_Agent(player_id=1, team_id=1),
               Greedy_Agent(player_id=2, team_id=0),
               Greedy_Agent(player_id=3, team_id=1)]
    
    env = JassEnv(players=players, print_globals=False)
    
    count_game_type = {
        "TOP_DOWN": 0,
        "BOTTOM_UP": 0,
        "ROSE": 0,
        "SCHILTE": 0,
        "EICHEL": 0,
        "SCHELLE": 0,
        "SCHIEBEN": 0
    }

    for _ in tqdm(range(n)):

        state = env.reset(starting_player_id=0)
        
        game_type = players[0].choose_game_type(state=state)
        
        if game_type == "TOP_DOWN":
            count_game_type["TOP_DOWN"] += 1
            continue
        elif game_type == "BOTTOM_UP":
            count_game_type["BOTTOM_UP"] += 1
            continue
        elif game_type == "ROSE":
            count_game_type["ROSE"] += 1
            continue
        elif game_type == "SCHILTE":
            count_game_type["SCHILTE"] += 1
            continue
        elif game_type == "EICHEL":
            count_game_type["EICHEL"] += 1
            continue
        elif game_type == "SCHELLE":
            count_game_type["SCHELLE"] += 1
            continue
        elif game_type == "SCHIEBEN":
            count_game_type["SCHIEBEN"] += 1
            continue
    
    game_type_probs = {game_type: count / n for game_type, count in count_game_type.items()}
    return game_type_probs


n = 1000000
game_type_probs = game_type_probabilities_dqn(n)

print(sum(game_type_probs.values()))
print(game_type_probs)

100%|██████████| 1000000/1000000 [06:45<00:00, 2463.72it/s]

1.0
{'TOP_DOWN': 0.037575, 'BOTTOM_UP': 0.150872, 'ROSE': 0.069505, 'SCHILTE': 0.073468, 'EICHEL': 0.080882, 'SCHELLE': 0.068887, 'SCHIEBEN': 0.518811}


In [4]:
model_path = "models/PI_dqn_128-128_500000.pt"
hide_opponents_hands = True
   
from tqdm import tqdm
from agents.dqn_agent import DQN_Agent
from agents.greedy_agent import Greedy_Agent
from envs.jassenv import JassEnv

def game_type_probabilities_dqn(n: int):
    
    agent = DQN_Agent(player_id=0, 
                        team_id=0,
                        deterministic=True,
                        hide_opponents_hands=hide_opponents_hands,
                        hidden_sizes=[128, 128],
                        activation="relu")
    agent.load_model(model_path)
    
    players = [agent,
               Greedy_Agent(player_id=1, team_id=1),
               Greedy_Agent(player_id=2, team_id=0),
               Greedy_Agent(player_id=3, team_id=1)]
    
    env = JassEnv(players=players, print_globals=False)
    
    count_game_type = {
        "TOP_DOWN": 0,
        "BOTTOM_UP": 0,
        "ROSE": 0,
        "SCHILTE": 0,
        "EICHEL": 0,
        "SCHELLE": 0,
        "SCHIEBEN": 0
    }

    for _ in tqdm(range(n)):

        state = env.reset(starting_player_id=0)
        
        game_type = players[0].choose_game_type(state=state)
        
        if game_type == "TOP_DOWN":
            count_game_type["TOP_DOWN"] += 1
            continue
        elif game_type == "BOTTOM_UP":
            count_game_type["BOTTOM_UP"] += 1
            continue
        elif game_type == "ROSE":
            count_game_type["ROSE"] += 1
            continue
        elif game_type == "SCHILTE":
            count_game_type["SCHILTE"] += 1
            continue
        elif game_type == "EICHEL":
            count_game_type["EICHEL"] += 1
            continue
        elif game_type == "SCHELLE":
            count_game_type["SCHELLE"] += 1
            continue
        elif game_type == "SCHIEBEN":
            count_game_type["SCHIEBEN"] += 1
            continue
    
    game_type_probs = {game_type: count / n for game_type, count in count_game_type.items()}
    return game_type_probs


n = 1000000
game_type_probs = game_type_probabilities_dqn(n)

print(sum(game_type_probs.values()))
print(game_type_probs)

100%|██████████| 1000000/1000000 [06:25<00:00, 2592.48it/s]

1.0
{'TOP_DOWN': 0.210337, 'BOTTOM_UP': 0.30699, 'ROSE': 0.043063, 'SCHILTE': 0.022591, 'EICHEL': 0.018677, 'SCHELLE': 0.035018, 'SCHIEBEN': 0.363324}
